# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


## Create workspace and experiment

In [13]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
experiment_name = 'capstone_project_hyperdrive'

experiment=Experiment(ws, experiment_name)

## Connect to compute cluster

In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "notebook240238"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster")
except ComputeTargetException:
    print("Creating new cluster")

    compute_config = AmlCompute.provisioning_configuration(
        vm_size = 'STANDARD_D2_V2',
        min_modes=0,
        max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
cluster.wait_for_completion(show_output=True)

Found existing cluster

Running


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
from azureml.core import Dataset

data_name = "Diabetes-predict"

data = Dataset.get_by_name(ws, name=data_name) 

data.to_pandas_dataframe()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
99997,Male,66.0,0,0,former,27.83,5.7,155,0
99998,Female,24.0,0,0,never,35.42,4.0,100,0


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [23]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        '--C': choice(0.1,0.5,1,5),
        '--max_iter': choice(50, 100, 500)
    }
)

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

src = ScriptRunConfig(source_directory='./',
                script='train_hyperdrive.py',
                compute_target=cluster,
                environment=sklearn_env)

#TODO: Create  hyperdrive config

hyperdrive_run_config = HyperDriveConfig(run_config=src,
                hyperparameter_sampling=param_sampling,
                policy=early_termination_policy,
                primary_metric_name='Accuracy',
                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                max_total_runs=10,
                max_concurrent_runs=4)

In [24]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [25]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_d73f2328-2a42-4a2a-a752-fc3b6e5d5298
Web View: https://ml.azure.com/runs/HD_d73f2328-2a42-4a2a-a752-fc3b6e5d5298?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-240238/workspaces/quick-starts-ws-240238&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-08-14T10:44:36.381464][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-08-14T10:44:36.9534839Z][SCHEDULER][INFO]Scheduling job, id='HD_d73f2328-2a42-4a2a-a752-fc3b6e5d5298_0' 
[2023-08-14T10:44:37.1318247Z][SCHEDULER][INFO]Scheduling job, id='HD_d73f2328-2a42-4a2a-a752-fc3b6e5d5298_1' 
[2023-08-14T10:44:37.2320905Z][SCHEDULER][INFO]Scheduling job, id='HD_d73f2328-2a42-4a2a-a752-fc3b6e5d5298_2' 
[2023-08-14T10:44:37.214681][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-08-14T10:44:37.4651277Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_d73f2328-2a42-4a2a-a

{'runId': 'HD_d73f2328-2a42-4a2a-a752-fc3b6e5d5298',
 'target': 'notebook240238',
 'status': 'Completed',
 'startTimeUtc': '2023-08-14T10:44:35.51369Z',
 'endTimeUtc': '2023-08-14T10:49:09.226294Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '4b5486e7-1b0e-4a34-9675-de790f959747',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '12',
  'score': '0.96075',
  'best_child_run_id': 'HD_d73f2328-2a42-4a2a-a752-fc3b6e5d5298_4',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_d73f2328-2a42-4a2a-a752-fc3b6e5d5298_4'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'az

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [26]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)

Run(Experiment: capstone_project_hyperdrive,
Id: HD_d73f2328-2a42-4a2a-a752-fc3b6e5d5298_4,
Type: azureml.scriptrun,
Status: Completed)


In [29]:
print(best_run.get_metrics())

{'Regularization Strength:': 1.0, 'Max iterations:': 500, 'Accuracy': 0.96075}


In [28]:
#TODO: Save the best model
best_run.register_model(model_name = 'best_hd_model.pkl', model_path='.')

Model(workspace=Workspace.create(name='quick-starts-ws-240238', subscription_id='3d1a56d2-7c81-4118-9790-f85d1acf0c77', resource_group='aml-quickstarts-240238'), name=best_hd_model.pkl, id=best_hd_model.pkl:1, version=1, tags={}, properties={})

## Shout down compute

In [ ]:
try:
    cluster.delete()
    cluster.wait_for_completion(show_output=True)
    print('Compute was deletes')

except:
    print('Compute already deleted')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

